In [222]:
import train_wrn_ebm as train
import argparse
import torch
from torch_geometric import utils
from torch_geometric.datasets import TUDataset
from torch_geometric import data

In [129]:
dataset = TUDataset(root = "molecule_dataset", name = "MUTAG")

In [130]:
maxNodes = 0
for i in range(len(dataset)):
    n = dataset[i].num_nodes
    if (n > maxNodes):
        maxNodes = n

print(maxNodes)

28


In [226]:
testMol = dataset[0]
s = testMol.edge_stores[0]
s

{'edge_index': tensor([[ 0,  0,  1,  1,  2,  2,  3,  3,  3,  4,  4,  4,  5,  5,  6,  6,  7,  7,
          8,  8,  8,  9,  9,  9, 10, 10, 11, 11, 12, 12, 12, 13, 13, 14, 14, 14,
         15, 16],
        [ 1,  5,  0,  2,  1,  3,  2,  4,  9,  3,  5,  6,  0,  4,  4,  7,  6,  8,
          7,  9, 13,  3,  8, 10,  9, 11, 10, 12, 11, 13, 14,  8, 12, 12, 15, 16,
         14, 14]]), 'x': tensor([[1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0

In [160]:
print(s['edge_index'].size())   # 2 x E edges
print(s['x'].size())            # N nodes x 7 node features
print(s['edge_attr'].size())    #E edges X 4 edge features

torch.Size([2, 38])
torch.Size([17, 7])
torch.Size([38, 4])


GOAL:
1) create ordered map of (edge_type, vector_idx)
2) create function to use map to identify vector location

In [139]:
# MAKE MAP
vector_map = dict()

pos_ctr = 0
for i in range(28):
    for j in range(i):
        vector_map[str(j) + "-" + str(i)] = pos_ctr
        pos_ctr += 1

In [249]:
# FUNCTION TO MAP EDGES TO VECTOR
def get_input_vector(input: data.Data):
    adj_vector = torch.zeros(size = (len(vector_map), 1))
    indices = input.edge_stores[0]['edge_index']
    features = input.edge_stores[0]['x']

    e1 = indices[0].tolist()
    e2 = indices[1].tolist()
    
    for i in range(len(e1)):
        key = str(e1[i]) + "-" + str(e2[i])
        if key in vector_map:
            idx = vector_map[key]
            adj_vector[idx] = 1


    feature_vec = torch.reshape(s['x'], shape = (-1,1))
    concat_vector = torch.cat((adj_vector, feature_vec))

    return concat_vector


In [250]:
# EXAMPLE
get_input_vector(dataset[0])

torch.Size([17, 7])


RuntimeError: The size of tensor a (119) must match the size of tensor b (378) at non-singleton dimension 0

GOAL:
- extract feature matrix from dataset
- create function that concatenates this data
- create function that uses both to handle each molecule

In [172]:
s

{'edge_index': tensor([[ 0,  0,  1,  1,  2,  2,  3,  3,  3,  4,  4,  4,  5,  5,  6,  6,  7,  7,
          8,  8,  8,  9,  9,  9, 10, 10, 11, 11, 12, 12, 12, 13, 13, 14, 14, 14,
         15, 16],
        [ 1,  5,  0,  2,  1,  3,  2,  4,  9,  3,  5,  6,  0,  4,  4,  7,  6,  8,
          7,  9, 13,  3,  8, 10,  9, 11, 10, 12, 11, 13, 14,  8, 12, 12, 15, 16,
         14, 14]]), 'x': tensor([[1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0

TypeError: cat() received an invalid combination of arguments - got (), but expected one of:
 * (tuple of Tensors tensors, int dim, *, Tensor out)
 * (tuple of Tensors tensors, name dim, *, Tensor out)


In [207]:
v1 = torch.zeros(size = (2, 1))
v2 = torch.zeros(size = (3, 1))
torch.cat((v1, v2), 0)

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.]])